In [1]:
import os
import rospkg
import numpy as np
import tf

# Create a RosPack object
rospack = rospkg.RosPack()

# Get the path to the package this script is in
package_path = rospack.get_path('hri_predict_ros')

# Define the path to the plots directory
plot_dir = os.path.join(package_path, 'plots')
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

# Specify which topics to read from the rosbag file
topic_names = ['/offline/zed/zed_node/body_trk/skeletons']

n_kpts = 18
TF_world_camera = [0.100575, -0.9304, 2.31042, 0.180663, 0.516604, 0.119341, 0.828395]

translation_world_camera = np.array(TF_world_camera[0:3])
quaternion_world_camera = np.array(TF_world_camera[3:7])

# Convert the quaternion to a rotation matrix
rotation_matrix_world_camera = tf.transformations.quaternion_matrix(quaternion_world_camera)

# Create a translation matrix
translation_matrix_world_camera = tf.transformations.translation_matrix(translation_world_camera)

# Combine the rotation and translation to get the transformation matrix from the world frame to the camera frame
cam_to_world_matrix = tf.transformations.concatenate_matrices(
    translation_matrix_world_camera,
    rotation_matrix_world_camera
)

human_meas_names = ['human_kp{}_{}'.format(i, suffix)
                    for i in range(n_kpts)
                    for suffix in ['x', 'y', 'z']]

# Define the frequency of the measurements for resampling
f = 20 # Hz
meas_dt = 1/f
freq_str = f'{meas_dt}S' # seconds

Import measurements from bag file

In [2]:
import rosbag

# Define the path to the bag directory
bag_dir = os.path.join(package_path, 'logs', 'bag')

bag_files = os.listdir(bag_dir)
bag_files = [os.path.join(bag_dir, bag_file) for bag_file in bag_files]

bag_data = {}
for bag_file in bag_files:
    with rosbag.Bag(bag_file, 'r') as bag:
        rows_list = []
        for topic, msg, t in bag.read_messages(topics=topic_names):
            row_dict = {}

            timestamp = t.to_sec()

            human_meas = np.full((1, n_kpts*3), np.nan)
            if topic == '/offline/zed/zed_node/body_trk/skeletons':
                skeleton_kpts = np.full((n_kpts, 3), np.nan)
                if msg.objects:                
                    for obj in msg.objects:
                        # Extract skeleton keypoints from message ([x, y, z] for each kpt)
                        kpts = np.array([[kp.kp] for kp in obj.skeleton_3d.keypoints])
                        kpts = kpts[:n_kpts] # select only the first n_kpts

                        skeleton_kpts = np.reshape(kpts, (n_kpts, 3)) # reshape to (n_kpts, 3)

                        # Convert keypoints to world frame
                        for i in range(n_kpts):
                            # Create a homogeneous coordinate for the keypoint position
                            kpt = np.array([skeleton_kpts[i][0],
                                            skeleton_kpts[i][1],
                                            skeleton_kpts[i][2],
                                            1])

                            # Transform the keypoint to the world frame using the transformation matrix
                            kpt_world = np.dot(cam_to_world_matrix, kpt)

                            skeleton_kpts[i][0] = kpt_world[0]
                            skeleton_kpts[i][1] = kpt_world[1]
                            skeleton_kpts[i][2] = kpt_world[2]
                    
                else:
                    skeleton_kpts = np.full(skeleton_kpts.shape, np.nan)

                # Update current human measurement vector
                human_meas = skeleton_kpts.flatten()

            row_dict.update({'timestamp': timestamp})
            row_dict.update({'human_meas': human_meas.flatten()})

            rows_list.append(row_dict)

    subject_id = bag_file.split('.')[0].split('simple_')[-1]
    bag_data[subject_id] = rows_list

Store measurement data in a dictionary of Pandas dataframes

In [3]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

measurement_data = {}
for subject, bag in bag_data.items():
    data = pd.DataFrame(bag, columns=['timestamp', 'human_meas'])

    # split columns into separate columns
    for c in data.columns.values:
        data = pd.concat([data, data.pop(c).apply(pd.Series).add_prefix(c+"_")], axis=1)

    # change column names
    data.columns = ['timestamp'] + human_meas_names

    # Make time index relative to the start of the recording
    # data['timestamp'] = data['timestamp'] - data['timestamp'][0]

    # Convert the 'timestamp' column to a TimeDeltaIndex
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')

    # Increase the timestamp by 2 hours to match the system time
    data['timestamp'] = data['timestamp'] + pd.Timedelta(hours=2)

    # Resample the DataFrame to a known frequency
    # resampled_data = data.resample(freq_str, on='timestamp').mean()
    #resampled_data = data.resample(freq_str, on='timestamp').first()
    measurement_data[subject] = data

for key, value in measurement_data.items():
    print(f'{key}: \t\t{value}')

sub_7: 		                          timestamp  human_kp0_x  human_kp0_y  human_kp0_z   
0     2024-05-29 15:59:50.991061760     0.806568     0.103624     1.568231  \
1     2024-05-29 15:59:51.034729216     0.808113     0.104849     1.566351   
2     2024-05-29 15:59:51.082635776     0.812446     0.109188     1.560914   
3     2024-05-29 15:59:51.116218880     0.815618     0.112558     1.557282   
4     2024-05-29 15:59:51.158306304     0.816228     0.114140     1.557340   
...                             ...          ...          ...          ...   
12049 2024-05-29 16:08:06.688208384          NaN          NaN          NaN   
12050 2024-05-29 16:08:06.730912768          NaN          NaN          NaN   
12051 2024-05-29 16:08:06.762553344          NaN          NaN          NaN   
12052 2024-05-29 16:08:06.795526144          NaN          NaN          NaN   
12053 2024-05-29 16:08:06.863832832          NaN          NaN          NaN   

       human_kp1_x  human_kp1_y  human_kp1_z  human_kp

In [ ]:
import plotly.express as px

example_data = measurement_data['sub_7']

fig = px.scatter(example_data, x=example_data.index, y=['human_kp4_y'])

fig.update_traces(marker=dict(size=3))
fig.update_layout(title='kp4_y', xaxis_title='Timestamp', yaxis_title='Value')
fig.update_xaxes(tickformat='%H:%M:%S')

fig.show()

Import task data from GUI logs

In [5]:
gui_dir = os.path.join(package_path, 'logs', 'gui_data')

# Get all the file names in the directory
gui_files = os.listdir(gui_dir)

gui_data = {}
for gui_file in gui_files:
    # Check if the file is a text file
    if gui_file.endswith('.txt'): # they are txt files, but structured as csv
        # Construct the file path
        file_path = os.path.join(gui_dir, gui_file)
        
        # Read the file as a dataframe
        df = pd.read_csv(file_path)
        
        # Add the dataframe to the dictionary using a portion of the file name as the key
        key = gui_file.split('.')[0].split('_')[-2:]
        key = '_'.join(key)
        gui_data[key] = df

Define ranges from gui data to split measurements in tasks

In [6]:
VELOCITIES = ['SLOW', 'MEDIUM', 'FAST']
TASK_NAMES = ['PICK-&-PLACE', 'WALKING', 'PASSING-BY']

trigger_data = {}
for subject, gui in gui_data.items():
    for velocity in VELOCITIES:
        for task in TASK_NAMES:
            trigger_data[(subject, velocity, task)] = gui.loc[(gui['Velocity'] == velocity) & (gui['Task_name'] == task)]

            # only keep the first and last timestamp of each trigger_data
            first_last_timestamps = trigger_data[(subject, velocity, task)].iloc[[0, -1]]['Timestamp'].values
            trigger_data[(subject, velocity, task)] = first_last_timestamps

for key, value in trigger_data.items():
    print(f'{key}: \t\t{value}')

('sub_7', 'SLOW', 'PICK-&-PLACE'): 		['2024-05-29 16:00:13.014' '2024-05-29 16:01:33.608']
('sub_7', 'SLOW', 'WALKING'): 		['2024-05-29 16:01:52.605' '2024-05-29 16:02:18.978']
('sub_7', 'SLOW', 'PASSING-BY'): 		['2024-05-29 16:02:36.198' '2024-05-29 16:03:13.502']
('sub_7', 'MEDIUM', 'PICK-&-PLACE'): 		['2024-05-29 16:03:40.351' '2024-05-29 16:04:36.349']
('sub_7', 'MEDIUM', 'WALKING'): 		['2024-05-29 16:04:51.878' '2024-05-29 16:05:12.766']
('sub_7', 'MEDIUM', 'PASSING-BY'): 		['2024-05-29 16:05:27.715' '2024-05-29 16:05:54.797']
('sub_7', 'FAST', 'PICK-&-PLACE'): 		['2024-05-29 16:06:16.835' '2024-05-29 16:06:45.974']
('sub_7', 'FAST', 'WALKING'): 		['2024-05-29 16:06:57.594' '2024-05-29 16:07:13.431']
('sub_7', 'FAST', 'PASSING-BY'): 		['2024-05-29 16:07:26.762' '2024-05-29 16:07:47.146']
('sub_9', 'SLOW', 'PICK-&-PLACE'): 		['2024-05-30 12:26:01.924' '2024-05-30 12:27:24.148']
('sub_9', 'SLOW', 'WALKING'): 		['2024-05-30 12:27:42.568' '2024-05-30 12:28:08.361']
('sub_9', 'SLOW', '

Define helper functions for Kalman Filtering

In [7]:
def get_near_psd(P, max_iter=10):

    eps = 1e-3  # Small positive jitter for regularization
    increment_factor = 10  # Factor to increase eps if needed
        
    def is_symmetric(A):
        return np.allclose(A, A.T)
                    
    def is_positive_definite(A):
        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False
        
    for _ in range(max_iter):
        if is_symmetric(P) and is_positive_definite(P):
            return P  # The matrix is suitable for Cholesky
    
        # Make P symmetric
        P = (P + P.T) / 2
    
        # Set negative eigenvalues to zero
        eigval, eigvec = np.linalg.eig(P)
        eigval[eigval < 0] = 0
        # add a jitter for strictly positive
        eigval += eps
    
        # Reconstruct the matrix
        P = eigvec.dot(np.diag(eigval)).dot(eigvec.T)

        # Force P to be real
        P = np.real(P)

        # Force off-diagonal elements to be zero => do it since the keypoints are all independent
        # P = np.diag(np.diag(P))
    
        # Check if P is now positive definite
        if is_positive_definite(P):
            return P
    
        # Increase regularization factor for the next iteration
        eps *= increment_factor
    
    raise ValueError("Unable to convert the matrix to positive definite within max iterations.")

Define parameters for the filters

In [8]:
from filterpy.kalman import UnscentedKalmanFilter, MerweScaledSigmaPoints
from filterpy.kalman import IMMEstimator
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
import copy, time
from tqdm import tqdm

# Parameters
dt = 0.1
predict_k_steps = True
pred_steps = 5          # [paper: n] number of prediction steps
n_kpts = 18
n_var_per_dof = 3       # position, velocity, acceleration
n_dim_per_kpt = 3       # x, y, z
var_r = 0.0025          # [paper: r_y] Hip: 3-sigma (99.5%) = 0.15 m ==> sigma 0.05 m ==> var = (0.05)^2 m^2
var_q = 0.1             # [paper: q_a] a_dot = u (u = 0 is very uncertain ==> add variance here)
var_P_pos = var_r       # [paper: p_y] Set equal to the measurement noise since the state is initialized with the measurement
var_P_vel = 0.02844     # [paper: p_v] Hip: no keypoint moves faster than 1.6 m/s ==> 3-sigma (99.5%) = 1.6 m/s ==> var = (1.6/3)^2 m^2/s^2
var_P_acc = 1.1111      # [paper: p_a] Hip: no keypoint accelerates faster than 10 m/s^2 ==> 3-sigma (99.5%) = 10 m/s^2 ==> var = (10/3)^2 m^2/s^4
max_time_no_meas = pd.Timedelta(seconds=1.0)

# Transition matrix for IMM
NUM_FILTERS_IN_BANK = 3
M = np.array([[0.55, 0.15, 0.30],
              [0.15, 0.75, 0.10],
              [0.60, 0.30, 0.10]])
mu = np.array([0.55, 0.40, 0.05])

# Dimensions
dim_x = n_var_per_dof * n_dim_per_kpt * n_kpts # 3D (position, velocity, acceleration) for each keypoint
dim_z = n_dim_per_kpt * n_kpts # 3D position for each keypoint

# Initial covariance matrix for all keypoints
def initialize_P(n_dim_per_kpt, n_kpts, var_P_pos, var_P_vel, var_P_acc):
    if n_var_per_dof == 1:
        init_P = np.eye(n_dim_per_kpt * n_kpts) * var_P_pos
    elif n_var_per_dof == 2:
        init_P = np.diag([var_P_pos, var_P_vel]) # initial state covariance for the single keypoint
        init_P = block_diag(*[init_P for _ in range(n_dim_per_kpt * n_kpts)])
    elif n_var_per_dof == 3:
        init_P = np.diag([var_P_pos, var_P_vel, var_P_acc]) # initial state covariance for the single keypoint
        init_P = block_diag(*[init_P for _ in range(n_dim_per_kpt * n_kpts)])
    else:
        raise ValueError('Invalid n_var_per_dof')
    return init_P
    
init_P = initialize_P(n_dim_per_kpt, n_kpts, var_P_pos, var_P_vel, var_P_acc)

# Position indices
p_idx = np.arange(0, dim_x, n_var_per_dof)

# Column names
state_names = ['kp{}_{}'.format(i, suffix)
               for i in range(n_kpts)
               for suffix in ['x', 'xd', 'xdd', 'y', 'yd', 'ydd', 'z', 'zd', 'zdd']]
measurement_names = ['kp{}_{}'.format(i, suffix)
                     for i in range(n_kpts)
                     for suffix in ['x', 'y', 'z']]
filtered_column_names = ['{}_kp{}_{}'.format(filt_type, i, suffix)
                         for filt_type in ['ca', 'cv', 'imm']
                         for i in range(n_kpts)
                         for suffix in ['x', 'xd', 'xdd', 'y', 'yd', 'ydd', 'z', 'zd', 'zdd']]
filtered_pred_column_names = ['{}_kp{}_{}'.format(filt_type, i, suffix)
                              for filt_type in ['ca', 'imm']
                              for i in range(n_kpts)
                              for suffix in ['x', 'xd', 'xdd', 'y', 'yd', 'ydd', 'z', 'zd', 'zdd']]
col_names_imm = ['imm_pos', 'imm_vel', 'imm_acc']
col_names_prob_imm = ['prob_ca', 'prob_ca_no', 'prob_cv']

# measurement function: only the position is measured
def hx(x):
    return x[p_idx]

sigmas = MerweScaledSigmaPoints(n=dim_x, alpha=.1, beta=2., kappa=1.)

# CONSTANT ACCELERATION UKF
F_block_ca = np.array([[1, dt, 0.5*dt**2],
                       [0, 1, dt],
                       [0, 0, 1]])
F_ca = block_diag(*[F_block_ca for _ in range(n_dim_per_kpt * n_kpts)])

# state transition function: const acceleration
def fx_ca(x, dt):
    return np.dot(F_ca, x)

# CONSTANT VELOCITY UKF
F_block_cv = np.array([[1, dt, 0],
                       [0, 1, 0],
                       [0, 0, 0]])
F_cv = block_diag(*[F_block_cv for _ in range(n_dim_per_kpt * n_kpts)])

# state transition function: const velocity
def fx_cv(x, dt):
    return np.dot(F_cv, x)

Execute predict-update-k_step_predict loop:
- for each subject
- for each velocity
- for each task

and aggregate results

In [9]:
def run_filtering_loop(subject_ids, velocities, task_names, pred_horizons, init_P, var_r, var_q):
    # CONSTANT ACCELERATION UKF
    ca_ukf = UnscentedKalmanFilter(dim_x=dim_x, dim_z=dim_z, dt=dt, hx=hx, fx=fx_ca, points=sigmas)
    ca_ukf.x = np.nan * np.ones(dim_x)
    ca_ukf.P = init_P
    ca_ukf.R = np.eye(dim_z)* var_r
    ca_ukf.Q = Q_discrete_white_noise(dim=n_var_per_dof, dt=dt, var=var_q, block_size=n_dim_per_kpt * n_kpts)

    # CONSTANT ACCELERATION UKF WITH NO PROCESS ERROR
    ca_no_ukf = copy.deepcopy(ca_ukf)
    ca_no_ukf.Q = np.zeros((dim_x, dim_x))

    # CONSTANT VELOCITY UKF
    cv_ukf = UnscentedKalmanFilter(dim_x=dim_x, dim_z=dim_z, dt=dt, hx=hx, fx=fx_cv, points=sigmas)
    cv_ukf.x = np.nan * np.ones(dim_x)
    cv_ukf.P = init_P
    cv_ukf.R = np.eye(dim_z)* var_r
    cv_ukf.Q = Q_discrete_white_noise(dim=n_var_per_dof, dt=dt, var=var_q, block_size=n_dim_per_kpt * n_kpts)
    
    # IMM ESTIMATOR
    filters = [copy.deepcopy(ca_ukf), ca_no_ukf, copy.deepcopy(cv_ukf)]

    bank = IMMEstimator(filters, mu, M)

    # K-STEP AHEAD PREDICTION FILTERS (declare k dictionaries to store the time series of predicted states and covariances)
    ca_ukf_pred = copy.deepcopy(ca_ukf)
    uxs_ca_pred = {}
    uxs_ca_pred_cov = {}
    bank_pred = copy.deepcopy(bank)
    uxs_bank_pred = {}
    uxs_bank_pred_cov = {}
    probs_bank_pred = {}

    # Create dictionary to store results
    measurement_split = {}   # dictionary of DataFrames with the measurements split by task
    filtering_results = {}   # dictionary of dictionaries with the filtering results
    prediction_results = {}  # dictionary of dictionaries with the k-step ahead prediction results

    for k in pred_horizons:
        for subject_id in subject_ids:
            for velocity in velocities:
                for task in task_names:
                    print(f'Processing {subject_id} - {velocity} - {task} for {k} steps ahead...')

                    # Reinitialize the lists to store the filtering results
                    uxs_ca, uxs_cv, uxs_bank, probs_bank = [], [], [], []
                    uxs_ca_cov, uxs_bank_cov = [], []

                    # Get the trigger timestamps for the current task
                    trigger_timestamps = trigger_data[(subject_id, velocity, task)]

                    # Get only the measurements whose timestamps are within the trigger timestamps
                    start_trigger = pd.to_datetime(trigger_timestamps[0])
                    end_trigger = pd.to_datetime(trigger_timestamps[1])

                    print("Selecting measurements from: ", start_trigger, "to", end_trigger)

                    zs = measurement_data[subject_id].loc[(measurement_data[subject_id]['timestamp'] >= start_trigger) &
                                                        (measurement_data[subject_id]['timestamp'] <= end_trigger)]
                    #zs.set_index('timestamp', inplace=True)

                    # Resample the measurements to a known frequency and subtract initial time
                    # zs = zs.resample(freq_str).mean()
                    #zs.index = zs.index - zs.index[0]
                    zs["timestamp"] = zs["timestamp"] - zs["timestamp"].iloc[0]
                    
                    measurement_split[(k, subject_id, velocity, task)] = zs

                    # Define times
                    t = zs["timestamp"].iloc[0]
                    t_end = zs["timestamp"].iloc[-1]
                    t_incr = pd.Timedelta(seconds=dt)

                    print("Start time:", t, "End time:", t_end)

                    # Initialization flag
                    time_no_meas = pd.Timedelta(seconds=0)
                    ufk_initialized = False
                    filt_timestamps = []
                    elapsed_time = 0.0

                    # Main loop
                    total_iterations = int((t_end - t) / t_incr) + 1
                    pbar = tqdm(total=total_iterations)

                    # Create dictionaries to store the k-step ahead prediction results
                    if predict_k_steps:
                        for i in range(k):
                            uxs_ca_pred[i] = []
                            uxs_bank_pred[i] = []
                            probs_bank_pred[i] = []
                            uxs_ca_pred_cov[i] = []
                            uxs_bank_pred_cov[i] = []

                    while t <= t_end:
                        tic = time.time()
                        filt_timestamps.append(t)
                        k_step_pred_executed = False

                        # Get the measurements in the current time window
                        tmp_db =zs.loc[(zs["timestamp"]>=t) & (zs["timestamp"]<=t+t_incr)]
                        measure_received = False
                        if (tmp_db.shape[0] > 0):
                            z = np.double(np.array(tmp_db.iloc[-1][1:])) # Select the last measurement in the time window
                            measure_received = not np.isnan(z).any() # Consider the measurement only if it is not NaN
                            
                        if measure_received and not ufk_initialized:
                            # print('timestamp:', t, 'measure:', z, 'initializing filters')
                            # initial state: [pos, vel, acc] = [current measured position, 0.0, 0.0]
                            ca_ukf.x = np.zeros(dim_x)
                            ca_ukf.x[p_idx] = z
                            cv_ukf.x = np.zeros(dim_x)
                            cv_ukf.x[p_idx] = z
                            for f in bank.filters:
                                f.x = np.zeros(dim_x)
                                f.x[p_idx] = z
                            ufk_initialized = True

                        else:
                            if not measure_received and ufk_initialized:
                                time_no_meas += t_incr
                                # print('timestamp:', t, 'no measure received for', time_no_meas, 'seconds')

                            if time_no_meas >= max_time_no_meas:
                                ufk_initialized = False
                            
                                # Reset filter states
                                ca_ukf.x = np.nan * np.ones(dim_x)
                                cv_ukf.x = np.nan * np.ones(dim_x)
                                bank.x = np.nan * np.ones(dim_x)
                                if predict_k_steps:
                                    ca_ukf_pred.x = np.nan * np.ones(dim_x)
                                    bank_pred.x = np.nan * np.ones(dim_x)

                                # Reset filter covariances
                                ca_ukf.P = init_P
                                cv_ukf.P = init_P
                                bank.P = init_P
                                if predict_k_steps:
                                    ca_ukf_pred.P = init_P
                                    bank_pred.P = init_P
                                
                            if ufk_initialized:
                                try:
                                    # make sure covariance matrices are positive semidefinite
                                    ca_ukf.P = get_near_psd(ca_ukf.P)
                                    cv_ukf.P = get_near_psd(cv_ukf.P)
                                    for f in bank.filters:
                                        f.P = get_near_psd(f.P)
                                    
                                    ca_ukf.predict()
                                    cv_ukf.predict()
                                    bank.predict()

                                    if measure_received:
                                        time_no_meas = pd.Timedelta(seconds=0)
                                        ca_ukf.update(z)
                                        cv_ukf.update(z)
                                        bank.update(z)

                                    if predict_k_steps:
                                        # Predict k steps ahead starting from the current state and covariance
                                        ca_ukf_pred.x = ca_ukf.x.copy()
                                        ca_ukf_pred.P = ca_ukf.P.copy()
                                        bank_pred.x = bank.x.copy()
                                        for f_pred, f in zip(bank_pred.filters, bank.filters):
                                            f_pred.x = f.x.copy()
                                            f_pred.P = f.P.copy()
                                            
                                        for i in range(k):
                                            # make sure covariance matrices are positive semidefinite
                                            ca_ukf_pred.P = get_near_psd(ca_ukf_pred.P)
                                            for f in bank_pred.filters:
                                                f.P = get_near_psd(f.P)

                                            ca_ukf_pred.predict()
                                            bank_pred.predict()

                                            uxs_ca_pred[i].append(ca_ukf_pred.x.copy())
                                            uxs_bank_pred[i].append(bank_pred.x.copy())
                                            probs_bank_pred[i].append(bank_pred.mu.copy())
                                            uxs_ca_pred_cov[i].append(ca_ukf_pred.P.copy().flatten())
                                            uxs_bank_pred_cov[i].append(bank_pred.P.copy().flatten())

                                        k_step_pred_executed = True

                                except np.linalg.LinAlgError as e:
                                    print(f"LinAlgError: {e}")

                                    # Reset filters
                                    ufk_initialized = False
                                        
                                    # Reset filter states
                                    ca_ukf.x = np.nan * np.ones(dim_x)
                                    cv_ukf.x = np.nan * np.ones(dim_x)
                                    bank.x = np.nan * np.ones(dim_x)
                                    if predict_k_steps:
                                        ca_ukf_pred.x = np.nan * np.ones(dim_x)
                                        bank_pred.x = np.nan * np.ones(dim_x)
                                        bank_pred.mu = np.nan * np.ones(NUM_FILTERS_IN_BANK) # IMM probabilities (3 filters)

                                    # Reset filter covariances
                                    ca_ukf.P = init_P
                                    cv_ukf.P = init_P
                                    bank.P = init_P
                                    if predict_k_steps:
                                        ca_ukf_pred.P = init_P
                                        bank_pred.P = init_P
                                
                        uxs_ca.append(ca_ukf.x.copy())
                        uxs_cv.append(cv_ukf.x.copy())
                        uxs_bank.append(bank.x.copy())
                        probs_bank.append(bank.mu.copy())
                        uxs_ca_cov.append(ca_ukf.P.copy().flatten())
                        uxs_bank_cov.append(bank.P.copy().flatten())

                        if not k_step_pred_executed:
                            for i in range(k):
                                uxs_ca_pred[i].append(ca_ukf.x.copy())
                                uxs_bank_pred[i].append(bank.x.copy())
                                probs_bank_pred[i].append(bank.mu.copy())
                                uxs_ca_pred_cov[i].append(ca_ukf.P.copy().flatten())
                                uxs_bank_pred_cov[i].append(bank.P.copy().flatten())

                        t += t_incr
                        toc = time.time()
                        elapsed_time += (toc - tic)
                        
                        pbar.update()

                    pbar.close()
                    print("Mean loop frequency: {:.2f} Hz".format(1.0 / (elapsed_time / len(filt_timestamps))))

                    # Create DataFrames with the filtered data
                    uxs_ca = np.array(uxs_ca)
                    uxs_cv = np.array(uxs_cv)
                    uxs_bank = np.array(uxs_bank)
                    uxs = np.concatenate((uxs_ca, uxs_cv, uxs_bank), axis=1)
                    probs_bank = np.array(probs_bank)
                    uxs_ca_cov = np.array(uxs_ca_cov)
                    uxs_bank_cov = np.array(uxs_bank_cov)
                    uxs_cov = np.concatenate((uxs_ca_cov, uxs_bank_cov), axis=1)

                    filtered_data = pd.DataFrame(uxs, index=filt_timestamps, columns=filtered_column_names)
                    imm_probs = pd.DataFrame(probs_bank, index=filt_timestamps, columns=col_names_prob_imm)
                    filtered_data_cov = pd.DataFrame(uxs_cov, index=filt_timestamps) # the elements of the flattened covariance matrices are stored in separate anonymous columns

                    if predict_k_steps:
                        kstep_pred_data = {}
                        kstep_pred_imm_probs = {}
                        kstep_pred_cov = {}

                        for i in range(k):
                            uxs_pred = np.concatenate((np.array(uxs_ca_pred[i]), np.array(uxs_bank_pred[i])), axis=1)
                            uxs_pred_cov = np.concatenate((np.array(uxs_ca_pred_cov[i]), np.array(uxs_bank_pred_cov[i])), axis=1)

                            kstep_pred_data[i] = pd.DataFrame(uxs_pred, index=filt_timestamps, columns=filtered_pred_column_names)
                            kstep_pred_imm_probs[i] = pd.DataFrame(np.array(probs_bank_pred[i]), index=filt_timestamps, columns=col_names_prob_imm)
                            kstep_pred_cov[i] = pd.DataFrame(uxs_pred_cov, index=filt_timestamps) # the elements of the flattened covariance matrices are stored in separate anonymous columns

                            # Shift the i-step ahead prediction data by i steps
                            kstep_pred_data[i] = kstep_pred_data[i].shift(+i)
                            kstep_pred_imm_probs[i] = kstep_pred_imm_probs[i].shift(+i)
                            kstep_pred_cov[i] = kstep_pred_cov[i].shift(+i)  

                    # Store filtering results
                    filtering_results[(k, subject_id, velocity, task)] = {
                        'filtered_data': filtered_data,
                        'imm_probs': imm_probs,
                        'filtered_data_cov': filtered_data_cov
                    }

                    # Store k-step prediction results
                    if predict_k_steps: 
                        prediction_results[(k, subject_id, velocity, task)] = {
                            'kstep_pred_data': kstep_pred_data,
                            'kstep_pred_imm_probs': kstep_pred_imm_probs,
                            'kstep_pred_cov': kstep_pred_cov
                        }

                    print(f"Processed {subject_id} - {velocity} - {task} for {k} steps ahead.")

    return measurement_split, filtering_results, prediction_results

In [10]:
pred_horizons = [pred_steps]
measurement_split, filtering_results, prediction_results = run_filtering_loop(['sub_8'], ['FAST'], ['PICK-&-PLACE'], pred_horizons,
                                                                              init_P, var_r, var_q)

Processing sub_8 - FAST - PICK-&-PLACE for 5 steps ahead...
Selecting measurements from:  2024-05-30 12:16:54.540000 to 2024-05-30 12:17:23.776000
Start time: 0 days 00:00:00 End time: 0 days 00:00:29.204252928


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 293/293 [03:06<00:00,  1.57it/s]


Mean loop frequency: 1.58 Hz
Processed sub_8 - FAST - PICK-&-PLACE for 5 steps ahead.


Plot some results

In [13]:
DEFAULT_PLOTLY_COLORS=['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                       'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
                       'rgb(148, 103, 189)', 'rgb(140, 86, 75)',
                       'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
                       'rgb(188, 189, 34)', 'rgb(23, 190, 207)']

# Add transparency to the colors
alpha = 0.2
DEFAULT_PLOTLY_COLORS_ALPHA = [color.replace('rgb', 'rgba').replace(')', f', {alpha})') for color in DEFAULT_PLOTLY_COLORS]

def plot_time_series(subject, velocity, task, kpt, dim, description, dim_type, k, n_var_per_dof, n_dim_per_kpt):
    state = 'kp{}_{}'.format(kpt, dim)
    state_idx = ['x', 'xd', 'xdd', 'y', 'yd', 'ydd', 'z', 'zd', 'zdd'].index(dim) + n_var_per_dof * n_dim_per_kpt * kpt
    variance_idx = dim_x * state_idx + state_idx

    if dim_type == 'pos':
        meas = measurement_split[(k, subject, velocity, task)].copy()
        meas_seconds = (meas["timestamp"] - meas["timestamp"].iloc[0]).dt.total_seconds()

    filt = filtering_results[(k, subject, velocity, task)]['filtered_data']
    filt_seconds = (filt.index - filt.index[0]).total_seconds()

    # Display the k-step ahead prediction (only last step)
    if predict_k_steps:
        kpred = prediction_results[(k, subject, velocity, task)]['kstep_pred_data'][k-1]
        kpred_seconds = (kpred.index - kpred.index[0]).total_seconds()
        kpred_variance = prediction_results[(k, subject, velocity, task)]['kstep_pred_cov'][k-1]

        # CA K-step ahead prediction
        std = kpred_variance.iloc[:, variance_idx].apply(np.sqrt)

        # Create pandas Series for the upper and lower confidence limits (1-sigma)
        kpred['_'.join(['ca', state, 'ucl'])] = kpred['_'.join(['ca', state])] + 1 * std
        kpred['_'.join(['ca', state, 'lcl'])] = kpred['_'.join(['ca', state])] - 1 * std

        # IMM K-step ahead prediction
        # (the index must be increased by the dimension of the flattened covariance matrix according to the way covariance matrices are stored)
        std = kpred_variance.iloc[:, variance_idx + dim_x*dim_x].apply(np.sqrt)

        # Create pandas Series for the upper and lower confidence limits (1-sigma)
        kpred['_'.join(['imm', state, 'ucl'])] = kpred['_'.join(['imm', state])] + 1 * std
        kpred['_'.join(['imm', state, 'lcl'])] = kpred['_'.join(['imm', state])] - 1 * std

    fig = px.line()
    if dim_type == 'pos':
        fig.add_scatter(x=meas_seconds,
                        y=meas['_'.join(('human',state))],
                        mode='lines+markers',
                        name='Measurements',
                        line=dict(color=DEFAULT_PLOTLY_COLORS[0])
        )
    fig.add_scatter(x=filt_seconds,
                    y=filt['_'.join(('ca',state))],
                    mode='lines+markers',
                    name='UKF CA',
                    line=dict(color=DEFAULT_PLOTLY_COLORS[1])
    )
    fig.add_scatter(x=filt_seconds,
                    y=filt['_'.join(('cv',state))],
                    mode='lines+markers',
                    name='UKF CV',
                    line=dict(color=DEFAULT_PLOTLY_COLORS[5])
    )
    fig.add_scatter(x=filt_seconds,
                    y=filt['_'.join(('imm',state))],
                    mode='lines+markers',
                    name='UKF IMM',
                    line=dict(color=DEFAULT_PLOTLY_COLORS[3])
    )
    if predict_k_steps:
        fig.add_scatter(x=kpred_seconds,
                        y=kpred['_'.join(('ca',state))],
                        mode='lines+markers',
                        name=f'CA {k}-step ahead prediction',
                        line=dict(color=DEFAULT_PLOTLY_COLORS[4])
        )
        fig.add_scatter(x=kpred_seconds,
                        y=kpred['_'.join(('imm',state))],
                        mode='lines+markers',
                        name=f'IMM {k}-step ahead prediction',
                        line=dict(color=DEFAULT_PLOTLY_COLORS[2])
        )
        fig.add_scatter(x=kpred_seconds,
                        y=kpred['_'.join(['ca', state, 'ucl'])],
                        mode='lines',
                        name=f'CA {k}-step ahead prediction (UCL)',
                        marker=dict(color=DEFAULT_PLOTLY_COLORS_ALPHA[4]),
                        line=dict(width=0),
                        fillcolor=DEFAULT_PLOTLY_COLORS_ALPHA[4],
        )
        fig.add_scatter(x=kpred_seconds,
                        y=kpred['_'.join(['ca', state, 'lcl'])],
                        mode='lines',
                        name=f'CA {k}-step ahead prediction (LCL)',
                        marker=dict(color=DEFAULT_PLOTLY_COLORS_ALPHA[4]),
                        line=dict(width=0),
                        fillcolor=DEFAULT_PLOTLY_COLORS_ALPHA[4],
                        fill='tonexty'
        )
        fig.add_scatter(x=kpred_seconds,
                        y=kpred['_'.join(['imm', state, 'ucl'])],
                        mode='lines',
                        name=f'IMM {k}-step ahead prediction (UCL)',
                        marker=dict(color=DEFAULT_PLOTLY_COLORS_ALPHA[2]),
                        line=dict(width=0),
                        fillcolor=DEFAULT_PLOTLY_COLORS_ALPHA[2]
        )
        fig.add_scatter(x=kpred_seconds,
                        y=kpred['_'.join(['imm', state, 'lcl'])],
                        mode='lines',
                        name=f'IMM {k}-step ahead prediction (LCL)',
                        marker=dict(color=DEFAULT_PLOTLY_COLORS_ALPHA[2]),
                        line=dict(width=0),
                        fillcolor=DEFAULT_PLOTLY_COLORS_ALPHA[2],
                        fill='tonexty'
        )

    fig.update_traces(marker=dict(size=2), line=dict(width=1))

    if dim_type == 'pos':
        fig.update_layout(title=description+" position"+f" [{subject}, {velocity}, {task}] "+f" (k={k})",
                          xaxis_title='Time (s)',
                          yaxis_title='Position (m)',
                          hovermode="x")
    elif dim_type == 'vel':
        fig.update_layout(title=description+" velocity"+f" [{subject}, {velocity}, {task}] "+f" (k={k})",
                          xaxis_title='Time (s)',
                          yaxis_title='Velocity (m/s)',
                          hovermode="x")
    elif dim_type == 'acc':
        fig.update_layout(title=description+" acceleration"+f" [{subject}, {velocity}, {task}] "+f" (dt={dt}, k={k})",
                          xaxis_title='Time (s)',
                          yaxis_title='Acceleration (m/s^2)',
                          hovermode="x")
    else:
        raise ValueError("Invalid dimension type. Use 'pos', 'vel', or 'acc'.")

    fig.show()

    # Save the plot to the plots folder in html format
    plot_name = '_'.join([subject, velocity, task, state, "dt", str(dt), "k", str(k), str(dim_type), "pred", str(predict_k_steps)])
    fig.write_html(os.path.join(plot_dir, plot_name+'.html')) # interactive plot
    fig.write_image(os.path.join(plot_dir, plot_name+ '.pdf')) # static plot

In [14]:
subject = 'sub_8'
velocity = 'FAST'
task = 'PICK-&-PLACE'
pred_horizon = 5
kpt = 2
dim = 'x' 
dim_type = 'pos' # ['pos', 'vel', 'acc'] # WORKING ?
description = 'Right-hand wrist keypoint x-coordinate'
# description = "Neck keypoint y-coordinate"

plot_time_series(subject, velocity, task, kpt, dim, description, dim_type, pred_horizon, n_var_per_dof, n_dim_per_kpt)

In [15]:
def plot_covariance_cone(subject, velocity, task, kpt, dim, description, dim_type, k, selected_timestamps, selected_range):
    state = 'kp{}_{}'.format(kpt, dim)
    state_idx = ['x', 'xd', 'xdd', 'y', 'yd', 'ydd', 'z', 'zd', 'zdd'].index(dim) + n_var_per_dof * n_dim_per_kpt * kpt
    variance_idx = dim_x * state_idx + state_idx

    if dim_type == 'pos':
        meas = measurement_split[(k, subject, velocity, task)].copy()
        meas_seconds = (meas["timestamp"] - meas["timestamp"].iloc[0]).dt.total_seconds()

    filt = filtering_results[(k, subject, velocity, task)]['filtered_data']
    filt_cov = filtering_results[(k, subject, velocity, task)]['filtered_data_cov']
    filt_seconds = (filt.index - filt.index[0]).total_seconds()

    fig = px.line()

    # Display the k-step ahead prediction (only last step)
    next_dataframes = {}
    for t in selected_timestamps:
        # Initialize lists to store the next k-step ahead states and confidence limits (1-sigma)
        std = np.sqrt(filt_cov.loc[t].iloc[variance_idx])
        next_k_states_ca = [filt.loc[t, '_'.join(['ca', state])]]
        next_k_lcls_ca = [filt.loc[t, '_'.join(['ca', state])] - 1 * std]
        next_k_ucls_ca = [filt.loc[t, '_'.join(['ca', state])] + 1 * std]
        next_k_states_imm = [filt.loc[t, '_'.join(['imm', state])]]
        next_k_lcls_imm = [filt.loc[t, '_'.join(['imm', state])] - 1 * std]
        next_k_ucls_imm = [filt.loc[t, '_'.join(['imm', state])] + 1 * std]
        times = [t]

        for step in range(k):
            # shift the timestamp by step*dt (had been shifted before for visualization purposes)
            t_shifted = t - pd.Timedelta(seconds=(step+1)*dt) 

            kpred = prediction_results[(k, subject, velocity, task)]['kstep_pred_data'][step]
            kpred_variance = prediction_results[(k, subject, velocity, task)]['kstep_pred_cov'][step]

            # CA K-step ahead prediction
            std = kpred_variance.iloc[:, variance_idx].apply(np.sqrt)

            # Create pandas Series for the upper and lower confidence limits (1-sigma)
            kpred['_'.join(['ca', state, 'ucl'])] = kpred['_'.join(['ca', state])] + 1 * std
            kpred['_'.join(['ca', state, 'lcl'])] = kpred['_'.join(['ca', state])] - 1 * std

            # IMM K-step ahead prediction
            # (the index must be increased by the dimension of the flattened covariance matrix according to the way covariance matrices are stored)
            std = kpred_variance.iloc[:, variance_idx + dim_x*dim_x].apply(np.sqrt)

            # Create pandas Series for the upper and lower confidence limits (1-sigma)
            kpred['_'.join(['imm', state, 'ucl'])] = kpred['_'.join(['imm', state])] + 1 * std
            kpred['_'.join(['imm', state, 'lcl'])] = kpred['_'.join(['imm', state])] - 1 * std

            t_shifted = t + pd.Timedelta(seconds=(step+1)*dt)

            # Store the next k-step ahead states and confidence limits
            next_k_states_ca.append(kpred.loc[t_shifted, '_'.join(['ca', state])])
            next_k_lcls_ca.append(kpred.loc[t_shifted, '_'.join(['ca', state, 'lcl'])])
            next_k_ucls_ca.append(kpred.loc[t_shifted, '_'.join(['ca', state, 'ucl'])])
            next_k_states_imm.append(kpred.loc[t_shifted, '_'.join(['imm', state])])
            next_k_lcls_imm.append(kpred.loc[t_shifted, '_'.join(['imm', state, 'lcl'])])
            next_k_ucls_imm.append(kpred.loc[t_shifted, '_'.join(['imm', state, 'ucl'])])
            
            # create array of timestamps from to t + k*dt
            times.append(t + pd.Timedelta(seconds=(step+1)*dt))

        # Transform the lists into Pandas DataFrames
        next_k_states_ca = pd.DataFrame(next_k_states_ca, index=times, columns=['CA'])
        next_k_lcls_ca = pd.DataFrame(next_k_lcls_ca, index=times, columns=['CA_LCL'])
        next_k_ucls_ca = pd.DataFrame(next_k_ucls_ca, index=times, columns=['CA_UCL'])
        next_k_states_imm = pd.DataFrame(next_k_states_imm, index=times, columns=['IMM'])
        next_k_lcls_imm = pd.DataFrame(next_k_lcls_imm, index=times, columns=['IMM_LCL'])
        next_k_ucls_imm = pd.DataFrame(next_k_ucls_imm, index=times, columns=['IMM_UCL'])

        next_dataframes[t] = {
            'next_k_states_ca': next_k_states_ca,
            'next_k_lcls_ca': next_k_lcls_ca,
            'next_k_ucls_ca': next_k_ucls_ca,
            'next_k_states_imm': next_k_states_imm,
            'next_k_lcls_imm': next_k_lcls_imm,
            'next_k_ucls_imm': next_k_ucls_imm
        }

        next_pred_ca = next_dataframes[t]['next_k_states_ca']['CA']
        next_pred_ca_lcl = next_dataframes[t]['next_k_lcls_ca']['CA_LCL']
        next_pred_ca_ucl = next_dataframes[t]['next_k_ucls_ca']['CA_UCL']
        next_pred_imm = next_dataframes[t]['next_k_states_imm']['IMM']
        next_pred_imm_lcl = next_dataframes[t]['next_k_lcls_imm']['IMM_LCL']
        next_pred_imm_ucl = next_dataframes[t]['next_k_ucls_imm']['IMM_UCL']

        next_pred_seconds = next_dataframes[t]['next_k_states_ca'].index.total_seconds()

        # Select time range before plotting between selected_range[0] and selected_range[1]
        meas_range = (meas['timestamp'] >= selected_range[0]) & (meas['timestamp'] <= selected_range[1])
        meas_cut = meas.loc[meas_range]
        filt_cut = filt.loc[selected_range[0]:selected_range[1]]
        kpred_cut = kpred.loc[selected_range[0]:selected_range[1]]

        filt_seconds = filt.index[(filt.index >= selected_range[0]) & (filt.index <= selected_range[1])].total_seconds()
        meas_seconds = meas_seconds[meas_range]
        kpred_seconds = filt_seconds

        fig.add_scatter(x=next_pred_seconds,
                        y=next_pred_ca,
                        mode='lines+markers',
                        name=f'CA next {k}-step at time {t}',
                        line=dict(color=DEFAULT_PLOTLY_COLORS[9]),
                        showlegend=False
        )
        fig.add_scatter(x=next_pred_seconds,
                        y=next_pred_imm,
                        mode='lines+markers',
                        name=f'IMM next {k}-step at time {t}',
                        line=dict(color=DEFAULT_PLOTLY_COLORS[7]),
                        showlegend=False
        )
        fig.add_scatter(x=next_pred_seconds,
                        y=next_pred_ca_lcl,
                        mode='lines',
                        name=f'CA next {k}-step at time {t} (LCL)',
                        marker=dict(color=DEFAULT_PLOTLY_COLORS_ALPHA[9]),
                        line=dict(width=0),
                        fillcolor=DEFAULT_PLOTLY_COLORS_ALPHA[9],
                        showlegend=False
        )
        fig.add_scatter(x=next_pred_seconds,
                        y=next_pred_ca_ucl,
                        mode='lines',
                        name=f'CA next {k}-step at time {t} (UCL)',
                        marker=dict(color=DEFAULT_PLOTLY_COLORS_ALPHA[9]),
                        line=dict(width=0),
                        fillcolor=DEFAULT_PLOTLY_COLORS_ALPHA[9],
                        fill='tonexty',
                        showlegend=False
        )
        fig.add_scatter(x=next_pred_seconds,
                        y=next_pred_imm_lcl,
                        mode='lines',
                        name=f'IMM next {k}-step at time {t} (LCL)',
                        marker=dict(color=DEFAULT_PLOTLY_COLORS_ALPHA[7]),
                        line=dict(width=0),
                        fillcolor=DEFAULT_PLOTLY_COLORS_ALPHA[7],
                        showlegend=False
        )
        fig.add_scatter(x=next_pred_seconds,
                        y=next_pred_imm_ucl,
                        mode='lines',
                        name=f'IMM next {k}-step at time {t} (UCL)',
                        marker=dict(color=DEFAULT_PLOTLY_COLORS_ALPHA[7]),
                        line=dict(width=0),
                        fillcolor=DEFAULT_PLOTLY_COLORS_ALPHA[7],
                        fill='tonexty',
                        showlegend=False
        )


    if dim_type == 'pos':
        fig.add_scatter(x=meas_seconds,
                        y=meas_cut['_'.join(('human',state))],
                        mode='lines+markers',
                        name='Measurements',
                        line=dict(color=DEFAULT_PLOTLY_COLORS[0])
        )
    fig.add_scatter(x=filt_seconds,
                    y=filt_cut['_'.join(('ca',state))],
                    mode='lines+markers',
                    name='UKF CA',
                    line=dict(color=DEFAULT_PLOTLY_COLORS[1])
    )
    fig.add_scatter(x=filt_seconds,
                    y=filt_cut['_'.join(('cv',state))],
                    mode='lines+markers',
                    name='UKF CV',
                    line=dict(color=DEFAULT_PLOTLY_COLORS[5])
    )
    fig.add_scatter(x=filt_seconds,
                    y=filt_cut['_'.join(('imm',state))],
                    mode='lines+markers',
                    name='UKF IMM',
                    line=dict(color=DEFAULT_PLOTLY_COLORS[3])
    )
    if predict_k_steps:
        fig.add_scatter(x=kpred_seconds,
                        y=kpred_cut['_'.join(('ca',state))],
                        mode='lines+markers',
                        name=f'CA {k}-step ahead prediction',
                        line=dict(color=DEFAULT_PLOTLY_COLORS[4])
        )
        fig.add_scatter(x=kpred_seconds,
                        y=kpred_cut['_'.join(('imm',state))],
                        mode='lines+markers',
                        name=f'IMM {k}-step ahead prediction',
                        line=dict(color=DEFAULT_PLOTLY_COLORS[2])
        )
        fig.add_scatter(x=kpred_seconds,
                        y=kpred_cut['_'.join(['ca', state, 'ucl'])],
                        mode='lines',
                        name=f'CA {k}-step ahead prediction (UCL)',
                        marker=dict(color=DEFAULT_PLOTLY_COLORS_ALPHA[4]),
                        line=dict(width=0),
                        fillcolor=DEFAULT_PLOTLY_COLORS_ALPHA[4],
        )
        fig.add_scatter(x=kpred_seconds,
                        y=kpred_cut['_'.join(['ca', state, 'lcl'])],
                        mode='lines',
                        name=f'CA {k}-step ahead prediction (LCL)',
                        marker=dict(color=DEFAULT_PLOTLY_COLORS_ALPHA[4]),
                        line=dict(width=0),
                        fillcolor=DEFAULT_PLOTLY_COLORS_ALPHA[4],
                        fill='tonexty'
        )
        fig.add_scatter(x=kpred_seconds,
                        y=kpred_cut['_'.join(['imm', state, 'ucl'])],
                        mode='lines',
                        name=f'IMM {k}-step ahead prediction (UCL)',
                        marker=dict(color=DEFAULT_PLOTLY_COLORS_ALPHA[2]),
                        line=dict(width=0),
                        fillcolor=DEFAULT_PLOTLY_COLORS_ALPHA[2]
        )
        fig.add_scatter(x=kpred_seconds,
                        y=kpred_cut['_'.join(['imm', state, 'lcl'])],
                        mode='lines',
                        name=f'IMM {k}-step ahead prediction (LCL)',
                        marker=dict(color=DEFAULT_PLOTLY_COLORS_ALPHA[2]),
                        line=dict(width=0),
                        fillcolor=DEFAULT_PLOTLY_COLORS_ALPHA[2],
                        fill='tonexty'
        )
        # Add two vertical lines, one at the first element of selected_timestamps and another 0.5 seconds after
        # (the line must touch the top and the bottom of the plot)
        fig.add_vrect(x0=selected_timestamps[0].total_seconds(),
                      x1=(selected_timestamps[0] + pd.Timedelta(seconds=0.5)).total_seconds(),
                      fillcolor="LightSalmon",
                      opacity=0.5,
                      layer="below",
                      line_width=0
        )
    # Add the horizontal arrow
    fig.add_annotation(
        x=((2*selected_timestamps[0] + pd.Timedelta(seconds=0.5)).total_seconds() / 2),  # This will center the arrow
        y=1.5,  # This will place the arrow on top of the plot
        text="",
        showarrow=True,
        arrowhead=1,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=-50, # This will make the arrow point to the right
        ay=0
    )

    fig.update_traces(marker=dict(size=2), line=dict(width=1))

    if dim_type == 'pos':
        fig.update_layout(title=description+" position"+f" [{subject}, {velocity}, {task}] "+f" (k={k})",
                        xaxis_title='Time (s)',
                        yaxis_title='Position (m)',
                        hovermode="x")
    elif dim_type == 'vel':
        fig.update_layout(title=description+" velocity"+f" [{subject}, {velocity}, {task}] "+f" (k={k})",
                        xaxis_title='Time (s)',
                        yaxis_title='Velocity (m/s)',
                        hovermode="x")
    elif dim_type == 'acc':
        fig.update_layout(title=description+" acceleration"+f" [{subject}, {velocity}, {task}] "+f" (dt={dt}, k={k})",
                        xaxis_title='Time (s)',
                        yaxis_title='Acceleration (m/s^2)',
                        hovermode="x")
    else:
        raise ValueError("Invalid dimension type. Use 'pos', 'vel', or 'acc'.")

    fig.show()

    # Save the plot to the plots folder in html format
    plot_name = '_'.join([subject, velocity, task, state, "dt", str(dt), "k", str(k), str(dim_type), "pred", str(predict_k_steps), "cone"])
    fig.write_html(os.path.join(plot_dir, plot_name+ '.html')) # interactive plot
    fig.write_image(os.path.join(plot_dir, plot_name+ '.pdf')) # static plot

Plot the covariance cone estimated at a given interval for k-steps ahead in time

In [16]:
# timestamp0 = pd.Timedelta(seconds=3.8)
# timestamp1 = pd.Timedelta(seconds=4.8)
# timestamp2 = pd.Timedelta(seconds=6.5)

# Define the time step to uniformly sample the time range
cone_step = 1

# Get the first and last timestamps of the measurements
start_meas = measurement_split[(pred_horizon, subject, velocity, task)].timestamp.iloc[0].total_seconds()
end_meas = measurement_split[(pred_horizon, subject, velocity, task)].timestamp.iloc[-1].total_seconds()

# Define a list of pd.Timedelta to uniformly sample the time range
uniform_timestamps = [pd.Timedelta(seconds=s) for s in np.arange(start_meas+dt*pred_steps, end_meas-cone_step, cone_step)]

# Define the time range to plot the covariance cones
lower_bound = pd.Timedelta(seconds=start_meas)
upper_bound = pd.Timedelta(seconds=end_meas)

# selected_timestamps = [timestamp0, timestamp1, timestamp2]
selected_timestamps = uniform_timestamps
selected_range = [lower_bound, upper_bound]

plot_covariance_cone(subject, velocity, task, kpt, dim, description, dim_type, pred_horizon, selected_timestamps, selected_range)

Tune parameters on training subjects.

In [17]:
# Subject IDS
subject_ids = gui_data.keys()
print("Subjects: ", subject_ids)

# Split subjects into train and test
train_subjects = ['sub_7', 'sub_8', 'sub_9'] # , 'sub_10', 'sub_13', 'sub_12', 'sub_3']
test_subjects = ['sub_9', 'sub_4', 'sub_11']

# Define which keypoints to consider for each task
keypoints = {'PICK-&-PLACE': [2, 3, 4, 5, 6, 7],
             'WALKING': [0, 1, 2, 5, 8, 11],
             'PASSING-BY': [0, 1, 2, 5, 8, 11]}

# Parameters
dt = 0.1
predict_k_steps = True
n_kpts = 18
n_var_per_dof = 3       # position, velocity, acceleration
n_dim_per_kpt = 3       # x, y, z
max_time_no_meas = pd.Timedelta(seconds=1.0)
NUM_FILTERS_IN_BANK = 3

# Initial values for the parameters
var_r = 0.0025          # [paper: r_y] Hip: 3-sigma (99.5%) = 0.15 m ==> sigma 0.05 m ==> var = (0.05)^2 m^2
var_q = 0.2             # [paper: q_a] a_dot = u (u = 0 is very uncertain ==> add variance here)
var_P_pos = var_r       # [paper: p_y] Set equal to the measurement noise since the state is initialized with the measurement
var_P_vel = 0.02844     # [paper: p_v] Hip: no keypoint moves faster than 1.6 m/s ==> 3-sigma (99.5%) = 1.6 m/s ==> var = (1.6/3)^2 m^2/s^2
var_P_acc = 1.1111      # [paper: p_a] Hip: no keypoint accelerates faster than 10 m/s^2 ==> 3-sigma (99.5%) = 10 m/s^2 ==> var = (10/3)^2 m^2/s^4

M = np.array([[0.55, 0.15, 0.30], # transition matrix for the IMM estimator
              [0.15, 0.75, 0.10],
              [0.60, 0.30, 0.10]])

mu = np.array([0.55, 0.40, 0.05]) # initial mode probabilities for the IMM estimator

# Tuning parameters
iter_P = 1
iter_q = 10
decrement_factor_q = 0.5
decrement_factor_P = 0.5

PRED_HORIZONS = [1, 3, 5]

Subjects:  dict_keys(['sub_7', 'sub_9', 'sub_13', 'sub_8', 'sub_12'])


Parameter var_q tuning.

In [19]:
# Compute the average error between the filtered states and the k-step ahead predictions
def compute_mean_error(filt, kpred, ca_states, imm_states):
    ca_error = np.mean(filt[ca_states] - kpred[ca_states])
    imm_error = np.mean(filt[imm_states] - kpred[imm_states])

    return ca_error, imm_error

# Compute the average standard deviation of the error between the filtered states and the k-step ahead predictions 
def compute_std_error(filt, kpred, ca_states, imm_states):
    ca_error = filt[ca_states] - kpred[ca_states]
    ca_error_mean = np.mean(ca_error)
    ca_error_std = np.std(ca_error - ca_error_mean)

    imm_error = filt[imm_states] - kpred[imm_states]
    imm_error_mean = np.mean(imm_error)
    imm_error_std = np.std(imm_error - imm_error_mean)
    
    # Average value for all states
    ca_error_std = np.mean(ca_error_std)
    imm_error_std = np.mean(imm_error_std)

    return ca_error_std, imm_error_std

# Compute the percentage of k-step ahead samples that fall within the band current filtered state +- 1*std
def compute_avg_perc(filt, kpred, kpred_var, ca_states, imm_states, ca_variance_idxs, imm_variance_idxs):
    ca_filtered_state = filt[ca_states]
    
    ca_std = kpred_var[ca_variance_idxs].apply(np.sqrt)
    ca_std.columns = ca_states
    ca_pred_lcl = kpred[ca_states] - 1 * ca_std
    ca_pred_ucl = kpred[ca_states] + 1 * ca_std

    ca_in_CI_band = (ca_filtered_state > ca_pred_lcl) & (ca_filtered_state < ca_pred_ucl)
    ca_perc = 100 * np.sum(ca_in_CI_band) / len(filt)

    imm_filtered_state = filt[imm_states]

    imm_std = kpred_var[imm_variance_idxs].apply(np.sqrt)
    imm_std.columns = imm_states
    imm_pred_lcl = kpred[imm_states] - 1 * imm_std
    imm_pred_ucl = kpred[imm_states] + 1 * imm_std

    imm_in_CI_band = (imm_filtered_state > imm_pred_lcl) & (imm_filtered_state < imm_pred_ucl)
    imm_perc = 100 * np.sum(imm_in_CI_band) / len(filt)

    return ca_perc, imm_perc
    

def parameter_tuning(training_subjects, velocities, tasks, pred_horizons, keypoints,
                     init_P, var_r, var_q, decrement_factor_q, decrement_factor_P, n_iter_P, n_iter_q,
                     n_var_per_dof, n_dim_per_kpt, dim_x, n_kpts,
                     var_P_pos, var_P_vel, var_P_acc):
    
    var_P_acc_init = var_P_acc
    
    for i in range(n_iter_q):
        for j in range(n_iter_P):
            print(f"Iteration {i+1}/{n_iter_q} for var_q and {j+1}/{n_iter_P} for var_P")
            print(f"Current values: var_q = {var_q}, var_P_acc = {var_P_acc}")

            init_P = initialize_P(n_dim_per_kpt, n_kpts, var_P_pos, var_P_vel, var_P_acc)

            _, filtering_results, prediction_results = run_filtering_loop(training_subjects, velocities, tasks, pred_horizons,
                                                                        init_P, var_r, var_q)

            # Compute the average error between the filtered states and the k-step ahead predictions
            avg_errors = {'PICK-&-PLACE': {'CA': 0, 'IMM': 0},
                        'WALKING': {'CA': 0, 'IMM': 0},
                        'PASSING-BY': {'CA': 0, 'IMM': 0}}
            
            # Compute the average standard deviation of the filtered states and the k-step ahead predictions
            avg_std = copy.deepcopy(avg_errors)
            
            # Compute the percentage of k-step ahead samples that fall within the band current filtered state +- 1*std
            avg_perc = copy.deepcopy(avg_errors)
            
            for k in pred_horizons:
                for subject_id in training_subjects:
                    for velocity in velocities:
                        for task in tasks:
                            filt = filtering_results[(k, subject_id, velocity, task)]['filtered_data']
                            kpred = prediction_results[(k, subject_id, velocity, task)]['kstep_pred_data'][k-1]
                            kpred_var = prediction_results[(k, subject_id, velocity, task)]['kstep_pred_cov'][k-1]

                            ca_states = ['ca_kp{}_{}'.format(kpt, dim)
                                        for dim in ['x', 'y', 'z']
                                        for kpt in keypoints[task]]
                            imm_states = ['imm_kp{}_{}'.format(kpt, dim)
                                        for dim in ['x', 'y', 'z']
                                        for kpt in keypoints[task]]
                            
                            ca_variance_idxs = []
                            for kpt in keypoints[task]:
                                for dim in ['x', 'y', 'z']:
                                    state_idx = ['x', 'xd', 'xdd', 'y', 'yd', 'ydd', 'z', 'zd', 'zdd'].index(dim) + n_var_per_dof * n_dim_per_kpt * kpt
                                    variance_idx = dim_x * state_idx + state_idx
                                    ca_variance_idxs.append(variance_idx)

                            imm_variance_idxs = []
                            for kpt in keypoints[task]:
                                for dim in ['x', 'y', 'z']:
                                    state_idx = ['x', 'xd', 'xdd', 'y', 'yd', 'ydd', 'z', 'zd', 'zdd'].index(dim) + n_var_per_dof * n_dim_per_kpt * kpt
                                    variance_idx = dim_x * state_idx + state_idx + dim_x * dim_x
                                    imm_variance_idxs.append(variance_idx)
                            
                            ca_error, imm_error = compute_mean_error(filt, kpred, ca_states, imm_states)
                            ca_std, imm_std = compute_std_error(filt, kpred, ca_states, imm_states)
                            ca_perc, imm_perc = compute_avg_perc(filt, kpred, kpred_var,
                                                                ca_states, imm_states, ca_variance_idxs, imm_variance_idxs)
                            
                            avg_errors[task]['CA'] += ca_error
                            avg_errors[task]['IMM'] += imm_error
                            avg_std[task]['CA'] += ca_std
                            avg_std[task]['IMM'] += imm_std
                            avg_perc[task]['CA'] += ca_perc
                            avg_perc[task]['IMM'] += imm_perc

            # Compute the average values of these aggregated metrics
            num_sums = len(pred_horizons) * len(training_subjects) * len(velocities) * len(tasks)
            avg_errors[task]['CA'] /= num_sums
            avg_errors[task]['IMM'] /= num_sums
            avg_std[task]['CA'] /= num_sums
            avg_std[task]['IMM'] /= num_sums
            avg_perc[task]['CA'] /= num_sums
            avg_perc[task]['IMM'] /= num_sums

            # Display results 
            # print("Average error:", avg_errors)
            # print("Average std:", avg_std)
            # print("Average percentage:", avg_perc)

            # Average over all selected keypoints
            avg_errors[task]['CA'] = np.mean(avg_errors[task]['CA'])
            avg_errors[task]['IMM'] = np.mean(avg_errors[task]['IMM'])
            avg_std[task]['CA'] = np.mean(avg_std[task]['CA'])
            avg_std[task]['IMM'] = np.mean(avg_std[task]['IMM'])
            avg_perc[task]['CA'] = np.mean(avg_perc[task]['CA'])
            avg_perc[task]['IMM'] = np.mean(avg_perc[task]['IMM'])

            # Display the results aggregating results for all keypoints
            print("Average error (CA): {:.6f}, {:.6f}, {:.6f}".format(
                avg_errors['PICK-&-PLACE']['CA'], avg_errors['WALKING']['CA'], avg_errors['PASSING-BY']['CA']))
            print("Average error (IMM): {:.6f}, {:.6f}, {:.6f}".format(
                avg_errors['PICK-&-PLACE']['IMM'], avg_errors['WALKING']['IMM'], avg_errors['PASSING-BY']['IMM']))
            print("Average std (CA): {:.4f}, {:.4f}, {:.4f}".format(
                avg_std['PICK-&-PLACE']['CA'], avg_std['WALKING']['CA'], avg_std['PASSING-BY']['CA']))
            print("Average std (IMM): {:.4f}, {:.4f}, {:.4f}".format(
                avg_std['PICK-&-PLACE']['IMM'], avg_std['WALKING']['IMM'], avg_std['PASSING-BY']['IMM']))
            print("Average percentage (CA): {:.4f}, {:.4f}, {:.4f}".format(
                avg_perc['PICK-&-PLACE']['CA'], avg_perc['WALKING']['CA'], avg_perc['PASSING-BY']['CA']))
            print("Average percentage (IMM): {:.4f}, {:.4f}, {:.4f}".format(
                avg_perc['PICK-&-PLACE']['IMM'], avg_perc['WALKING']['IMM'], avg_perc['PASSING-BY']['IMM']))
            print("===============================================\n\n")

            # Update the init_P parameter
            var_P_acc *= decrement_factor_P

        # Update the var_q parameter
        var_q *= decrement_factor_q

        # Reset the var_P_acc parameter to its initial value
        var_P_acc = var_P_acc_init

In [ ]:
tic = time.time()
train_subjects = train_subjects # ['sub_7'] 
velocities = ['FAST']
tasks = TASK_NAMES # ['PICK-&-PLACE']
pred_horizons = [5] # PRED_HORIZONS
parameter_tuning(train_subjects, ['FAST'], tasks, pred_horizons, keypoints, 
                 init_P, var_r, var_q, decrement_factor_q, decrement_factor_P, iter_P, iter_q,
                 n_var_per_dof, n_dim_per_kpt, dim_x, n_kpts,
                 var_P_pos, var_P_vel, var_P_acc)
toc = time.time()

minutes, seconds = divmod(toc - tic, 60)
print(f"Parameter tuning took {minutes:.0f} minutes and {seconds:.2f} seconds.")

Iteration 1/10 for var_q and 1/1 for var_P
Current values: var_q = 0.2, var_P_acc = 1.1111
Processing sub_7 - FAST - PICK-&-PLACE for 5 steps ahead...
Selecting measurements from:  2024-05-29 16:06:16.835000 to 2024-05-29 16:06:45.974000
Start time: 0 days 00:00:00 End time: 0 days 00:00:29.074720


 92%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊              | 267/291 [01:49<00:15,  1.58it/s]